In [1]:
#Import libraries
import pandas as pd
#use popular binance exchange API
from binance.client import Client
api_key ='EYLz9h7MwAVxmA36GfF8xgEOnds4AuAD3jLRbGEhzuQHwoLlMw7YEmMKR9hGPMUf'
#Do not share secret api key with anyone. The below key is a dummy one and will not work
api_secret = 't10mZ5hTRw3Sjjk3MKa1VkcDVUmbsVgtUXa3JKw23IFvrL857ioGxCsCyySbEy5X'
client = Client(api_key, api_secret)
print("logged in")

logged in


In [2]:
#Import raw data for functions to read later

In [25]:
raw_trade = pd.read_excel('Export Recent Trade History.xlsx')
raw_trade

,Date(UTC),Market,Type,Price,Amount,Total,Fee,Fee Coin
0,2021-02-28 15:32:26,ETHGBP,BUY,950.00,0.308270,292.856500,0,ETH
1,2021-02-28 15:15:30,BTCGBP,BUY,31940.00,0.000548,17.503120,0,BTC
2,2021-02-28 15:15:27,BTCGBP,BUY,31940.00,0.002582,82.469080,0,BTC
3,2021-02-20 11:14:36,BTCGBP,BUY,40064.00,0.004902,196.393728,0,BTC
4,2021-02-08 16:17:12,ETHGBP,BUY,1288.50,0.114320,147.301320,0,ETH
5,2021-02-08 11:04:36,ETHGBP,BUY,1212.14,0.000990,1.200019,0,ETH
6,2021-02-08 11:04:35,ETHGBP,BUY,1212.14,0.080000,96.971200,0,ETH


In [4]:
#Create function to fetch live exchange rate of all crytocurrencies in raw_trade data

In [17]:
def get_live_prices():
    #Uses client api to get all exchange rates
    prices = client.get_all_tickers()
    exchange_GBP=[]
    #Create unique list of all crytocurrenccies in raw_trade data
    currency = sorted(raw_trade['Fee Coin'].unique())
    #Use nested for loops to find the price of all symbols that equal an exchange code e.g. 'BTCGCP'
    for p in prices:
        for coin in range(len(currency)):
            if currency[coin]+'GBP' == p['symbol']: 
                exchange_GBP.append(float(p['price']))
    return exchange_GBP

In [26]:
live_prices=get_live_prices()

In [19]:
#Create function to transform raw_trade data into a final tracker dataframe giving P/L for aggregate cryptocurrencies

In [55]:
def transformer():
    export_history = raw_trade.copy()
    #Create unique list of all crytocurrenccies in raw_trade data
    currency = sorted(raw_trade['Fee Coin'].unique())
    #Create initial DF with single coin column
    data = {'Coin': currency}
    df = pd.DataFrame (data)
    
    #Sum all quantities of coin bought, grouped by coin
    df['Quantity']= export_history.groupby('Fee Coin',as_index=False)['Amount'].sum().iloc[:,-1]
    #Sum all values of coin bought, grouped by coin
    df['Money Spent']= export_history.groupby('Fee Coin',as_index=False)['Total'].sum().iloc[:,-1]
    
    #Add live prices generated and executed from get_live_prices() function
    df['Live price per coin'] = live_prices
    #Current value of each coin is price per coin * quantity' of coin
    df['Current Value']=df["Live price per coin"]*df['Quantity']
    #Profit/loss calculated by current value of the coin minus initial investment
    df['P/L']=df['Current Value'] - df['Money Spent']
    
    #Checkpoint
    df_pre = df.copy()

    #Sums all columns
    total_cols = df_pre.sum(axis=0)
    #Add to dataframe
    df_pre = df.append(total_cols, ignore_index=True)

    #P/L expressed as a percentage
    df_pre['P/L %']= (df_pre['Current Value'] - df_pre['Money Spent'])*100/df_pre['Money Spent']
    df_reorder = df_pre.copy()
    #Reorder columns
    df_reorder
    df_reorder=df_reorder[['Coin', 'Current Value',
       'P/L', 'P/L %',  'Money Spent', 'Quantity','Live price per coin' ]]
    
    #len(currency) is used as an index to give the bottom position of the dataframe
    #Rename the 1st column, bottom positon as 'Total' instead of all the coin names being concatenated together
    df_reorder.iloc[len(currency), df_reorder.columns.get_loc('Coin')] = 'Total'
    #Adding all coin currencies together doesn't make sense due to different units, hence N/A is added
    df_reorder.iloc[len(currency), df_reorder.columns.get_loc('Quantity')] = 'N/A'
    #Same logic applies for Live price per coin
    df_reorder.iloc[len(currency), df_reorder.columns.get_loc('Live price per coin')] = 'N/A' 
    
    #Checkpoint
    df_totals = df_reorder.copy()
    #Sort final dataframe to give Total row at the top and reset index
    df_tracker=df_totals.sort_values(by=['P/L'],ascending=False)
    df_tracker = df_tracker.round(2)
    return df_tracker

In [56]:
transformer()

,Coin,Current Value,P/L,P/L %,Money Spent,Quantity,Live price per coin
2,Total,1717.44,882.75,105.76,834.69,N/A,N/A
1,ETH,1379.59,841.26,156.27,538.33,0.50358,2739.56
0,BTC,337.86,41.49,14.00,296.37,0.008032,42063.8
